In [ ]:
!pip install --upgrade numpy
!pip uninstall tensorflow -y
!pip uninstall mediapipe-model-maker -y
!pip install tensorflow
!pip install mediapipe-model-maker

  Using cached numpy-2.2.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
Using cached numpy-2.2.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.4 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-model-optimization 0.8.0 requires numpy~=1.23, but you have numpy 2.2.4 which is incompatible.
mediapipe 0.10.21 requires numpy<2, but you have numpy 2.2.4 which is incompatible.
tensorflow 2.12.0 requires numpy<1.24,>=1.22, but you have numpy 2.2.4 which is incompatible.
tf-models-official 2.19.1 requires tensorflow~=2.19.0, but you have tensorflow 2.12.0 which is incompatible.
google-colab 1.0.0 requires google-auth==2.38.0, but you have google-auth 2.39.0 whi

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab import files
import os
import tensorflow as tf
assert tf.__version__.startswith('2')

from mediapipe_model_maker import gesture_recognizer

import matplotlib.pyplot as plt

/usr/local/lib/python3.11/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [ ]:
dataset_path = "/content/drive/MyDrive/SIBI/training"

In [ ]:
print(dataset_path)
labels = []
for i in os.listdir(dataset_path):
  if os.path.isdir(os.path.join(dataset_path, i)):
    labels.append(i)

labels.sort() #sort path from A-Z
print(labels)

/content/drive/MyDrive/SIBI/training
['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'None', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']


In [ ]:
NUM_EXAMPLES = 5
for label in labels:
  label_dir = os.path.join(dataset_path, label)
  example_filenames = os.listdir(label_dir)[:NUM_EXAMPLES]
  # Check if example_filenames is empty before proceeding
  if not example_filenames:
    print(f"No image files found in directory: {label_dir}")
    continue  # Skip to the next label if no files are found
  fig, axs = plt.subplots(1, NUM_EXAMPLES, figsize=(10, 2))
  for i in range(NUM_EXAMPLES):
    axs[i].imshow(plt.imread(os.path.join(label_dir, example_filenames[i])))
    axs[i].get_xaxis().set_visible(False)
    axs[i].get_yaxis().set_visible(False)
  fig.suptitle(f'Showing {NUM_EXAMPLES} examples for {label}')

plt.show()

No image files found in directory: /content/drive/MyDrive/SIBI/training/None


In [ ]:
data = gesture_recognizer.Dataset.from_folder(
    dirname=dataset_path,
    hparams=gesture_recognizer.HandDataPreprocessingParams()
)

train_data, rest_data = data.split(0.8)
validation_data, test_data = rest_data.split(0.5)

In [ ]:
hparams = gesture_recognizer.HParams(epochs = 250, batch_size = 16, export_dir="exported_model")

In [ ]:
options = gesture_recognizer.GestureRecognizerOptions(hparams=hparams)

In [ ]:
model = gesture_recognizer.GestureRecognizer.create(
      train_data = train_data,
      validation_data = validation_data,
      options = options
)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hand_embedding (InputLayer  [(None, 128)]             0         
 )                                                               
                                                                 
 batch_normalization (Batch  (None, 128)               512       
 Normalization)                                                  
                                                                 
 re_lu (ReLU)                (None, 128)               0         
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 custom_gesture_recognizer_  (None, 27)                3483      
 out (Dense)                                                     
                                                             

In [ ]:
model.export_model()
!ls exported_model

Using existing files at /tmp/model_maker/gesture_recognizer/palm_detection_full.tflite
Using existing files at /tmp/model_maker/gesture_recognizer/hand_landmark_full.tflite
best_model_weights.data-00000-of-00001	epoch_models		 metadata.json
best_model_weights.index		gesture_recognizer.task
checkpoint				logs


In [ ]:
loss, acc = model.evaluate(test_data, batch_size=16)
print(f"[INFO] accuracy: {acc * 100:.2f}%")  # Gunakan f-string dan format angka
print(f"[INFO] Loss: {loss:.4f}")

42/42 [==============================] - 6s 11ms/step - loss: 0.0020 - categorical_accuracy: 0.9985
[INFO] accuracy: 99.85%
[INFO] Loss: 0.0020


In [ ]:
!zip -r /content/exported_model_sibi.zip /content/exported_model

  adding: content/exported_model/ (stored 0%)
  adding: content/exported_model/logs/ (stored 0%)
  adding: content/exported_model/logs/validation/ (stored 0%)
  adding: content/exported_model/logs/validation/events.out.tfevents.1745056964.e785e8024f4b.16054.1.v2 (deflated 82%)
  adding: content/exported_model/logs/train/ (stored 0%)
  adding: content/exported_model/logs/train/events.out.tfevents.1745056958.e785e8024f4b.16054.0.v2 (deflated 81%)
  adding: content/exported_model/epoch_models/ (stored 0%)
  adding: content/exported_model/epoch_models/model-0164.data-00000-of-00001 (deflated 6%)
  adding: content/exported_model/epoch_models/model-0127.data-00000-of-00001 (deflated 6%)
  adding: content/exported_model/epoch_models/model-0081.data-00000-of-00001 (deflated 6%)
  adding: content/exported_model/epoch_models/model-0064.index (deflated 55%)
  adding: content/exported_model/epoch_models/model-0123.data-00000-of-00001 (deflated 6%)
  adding: content/exported_model/epoch_models/mode